
# 6.3.4 - Next-Gen Fine-Tuning with PEFT (DoRA)

This notebook demonstrates how to fine-tune a large language model using parameter-efficient methods such as DoRA (Decomposed and Recomposed Attention).

We will:
- Load a model and dataset
- Use the `peft` library with `LoraConfig`
- Enable DoRA for efficient adaptation
- Train using the `trl` library (SFTTrainer)

> Note: This example uses a small model for demonstration. Replace with larger models in production.


In [ ]:

!pip install transformers datasets peft trl accelerate


In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer
import torch


In [ ]:

dataset = load_dataset("squad")
sample = dataset["train"].select(range(100))  # use small subset


In [ ]:

model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [ ]:

def format_qa(example):
    question = example["question"]
    context = example["context"]
    answer = example["answers"]["text"][0]
    return {"text": f"question: {question} context: {context} answer: {answer}"}

formatted_dataset = sample.map(format_qa)


In [ ]:

peft_config = LoraConfig(
    r=64,
    target_modules=["q_proj", "v_proj"],
    use_dora=True,
    task_type="CAUSAL_LM"
)


In [ ]:

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    args={
        "per_device_train_batch_size": 4,
        "num_train_epochs": 1,
        "logging_steps": 10,
        "output_dir": "./dora-peft-model"
    }
)


In [ ]:

trainer.train()


In [ ]:

model.save_pretrained("./dora-peft-model")
tokenizer.save_pretrained("./dora-peft-model")


In [ ]:

prompt = "question: What is the capital of Germany? context: Germany is a country in Europe. Its capital is Berlin. answer:"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
output_ids = model.generate(input_ids, max_length=64)
print("Answer:", tokenizer.decode(output_ids[0], skip_special_tokens=True))
